In [1]:
import os
import pandas as pd
import numpy as np
import torch
from random import shuffle, sample
from torchvision import transforms
from PIL import Image
from torch.nn import functional as F
import matplotlib.pyplot as plt

import torch.nn as nn

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import SubsetRandomSampler, ConcatDataset
from torch.utils.data import random_split

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 

from torch.utils.data import Dataset 

In [2]:
dataPath = "action_recognition_dataset"
tools = ["spatula","ruler","hook","sshot"]
actions = ["left_to_right","pull","push","right_to_left"]
objects = ['woodenCube', 'tomatoCan', 'boxMilk', 'containerNuts', 'cornCob', 'yellowFruitToy', 'bottleNailPolisher', 'boxRealSense', 'clampOrange', 'greenRectangleToy', 'ketchupToy', 'peartoy', 'yogurtYellowbottle', 'cowToy', 'tennisBallYellowGreen', 'blackCoinBag', 'lemonSodaCan', 'peperoneGreenToy', 'boxEgg', 'pumpkinToy']


In [3]:
def openImageC(path):
    image = None
    transform = transforms.Compose([transforms.Resize((64,64)),
                                     transforms.ToTensor()
                                    ])
    with Image.open(path) as im:
        image=im.convert("RGB")
        image= transform(image)
        mean= image.mean([1,2])
        std = image.std([1,2])
        image = transforms.Normalize(mean,std)(image)
    return image

def openImageD(path):
    image = None
    transform = transforms.Compose([transforms.Resize((64,64)),
                                     transforms.ToTensor()
                                    ])
    with Image.open(path) as im:
        image=im.convert("RGB")
        image= transform(image)
        mean= image.mean([1,2])
        std = image.std([1,2])
        if std[0]==0:
            std[0]=1
        image = transforms.Normalize(mean,std)(image)
    return image

def setDataframe(path, tools, actions):
    dataset = pd.DataFrame()
    directories = os.listdir(path)
    
    
    for iFolder in range(len(directories)):
        objs = directories[iFolder]
        obj = objs.split("_")[1]
        objects.append(obj)
        for act in actions:
            for tool in tools:
                for i in range(10):
                    path = os.path.join(dataPath,objs,tool,act)
                    initColor = openImageC(os.path.join(path,"color",f"init_color_{i}.png"))
                    initDepth = openImageD(os.path.join(path,"depthcolormap",f"init_depthcolormap_{i}.png"))
                    effectColor = openImageC(os.path.join(path,"color",f"effect_color_{i}.png"))
                    effectDepth = openImageD(os.path.join(path,"depthcolormap",f"effect_depthcolormap_{i}.png"))
                    
                    frames = [obj,tool,act,initColor,effectColor,initDepth,effectDepth]
                    dataset=pd.concat([dataset,pd.Series(frames).to_frame().T],ignore_index=True)
    
    dataset = dataset.rename(columns ={0: "Object" ,1: "Tool",2:"Action", 3:"Color Initial",4:"Color Effect",5:"Depthcolor Initial",6:"Depthcolor Effect"})
    return dataset
        
def label_encode(entries, labels):
    dictL = dict()
    res = list()
    for i, label in enumerate(labels):
        dictL[label] = i
    
    for entrie in entries:
        res.append(dictL[entrie])
    
    return pd.Series(res)

In [4]:
dataset = setDataframe(dataPath,tools,actions)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'action_recognition_dataset'

In [ ]:

dataset["ActionL"] = label_encode(dataset["Action"],actions)
dataset["ToolL"]   = label_encode(dataset["Tool"],tools)



 # Late Fusion Model

In [ ]:
def build(numOutputC, n_layerC, numOutputH, n_layerH):
    model=nn.Sequential()
    model.add_module("Conv0",nn.Conv2d(3, numOutputC[0], kernel_size = 3, padding = 1))
    model.add_module("Relu0",nn.ReLU())
    model.add_module("Norm0",nn.BatchNorm2d(numOutputC[0]))
    model.add_module("MaxPool0",nn.MaxPool2d(2,2))
    
    i=0
    while i < n_layerC-1:
        model.add_module(f"Conv{i+1}",nn.Conv2d(numOutputC[i], numOutputC[i+1], kernel_size = 3, padding = 1))
        model.add_module(f"Relu{i+1}",nn.ReLU())
        model.add_module(f"Norm{i+1}",nn.BatchNorm2d(numOutputC[i+1]))
        model.add_module(f"MaxPool{i+1}",nn.MaxPool2d(2,2))
        i+=1
        
    model.add_module("Flatten", nn.Flatten())

#(2**(18-n_layerC)

    model.add_module("Drop",nn.Dropout(0.4))
    model.add_module("Linear",nn.Linear(2**(17-n_layerC),numOutputH[0]))
    model.add_module("ReluH0",nn.ReLU())
    model.add_module("NormH0",nn.BatchNorm1d(numOutputH[0]))
    i=0
    while i< n_layerH-1:
        
        model.add_module(f"Drop{i+1}",nn.Dropout(0.4))
        model.add_module(f"Linear{i+1}",nn.Linear(numOutputH[i], numOutputH[i+1]))
        model.add_module(f"ReluH{i+1}",nn.ReLU())
        model.add_module(f"NormH{i+1}",nn.BatchNorm1d(numOutputH[i+1]))
        i+=1
    return model

def buildD(numOutputC, n_layerC, numOutputH, n_layerH):
    model=nn.Sequential()
    model.add_module("Conv0",nn.Conv2d(3, numOutputC[0], kernel_size = 3, padding = 1))
    model.add_module("Relu0",nn.LeakyReLU(0.3))
    model.add_module("Norm0",nn.BatchNorm2d(numOutputC[0]))
    model.add_module("MaxPool0",nn.MaxPool2d(2,2))
    
    i=0
    while i < n_layerC-1:
        model.add_module(f"Conv{i+1}",nn.Conv2d(numOutputC[i], numOutputC[i+1], kernel_size = 3, padding = 1))
        model.add_module(f"Relu{i+1}",nn.LeakyReLU(0.3))
        model.add_module(f"Norm{i+1}",nn.BatchNorm2d(numOutputC[i+1]))
        model.add_module(f"MaxPool{i+1}",nn.MaxPool2d(2,2))
        i+=1
        
    model.add_module("Flatten", nn.Flatten())



    model.add_module("Drop",nn.Dropout(0.4))
    model.add_module("Linear",nn.Linear(2**(17-n_layerC),numOutputH[0]))
    model.add_module("ReluH0",nn.LeakyReLU(0.3))
    model.add_module("NormH0",nn.BatchNorm1d(numOutputH[0]))
    i=0
    while i< n_layerH-1:
        
        model.add_module(f"Drop{i+1}",nn.Dropout(0.4))
        model.add_module(f"Linear{i+1}",nn.Linear(numOutputH[i], numOutputH[i+1]))
        model.add_module(f"ReluH{i+1}",nn.LeakyReLU(0.3))
        model.add_module(f"NormH{i+1}",nn.BatchNorm1d(numOutputH[i+1]))
        i+=1
    return model


class ImageLateFClassifier(nn.Module):
    def __init__(self, numOutputC, n_layerC, numOutputH, n_layerH, depth):
        super().__init__()
        
        model = None
        if(depth):
            model = buildD(numOutputC, n_layerC, numOutputH, n_layerH)
        else:
            model = build(numOutputC, n_layerC, numOutputH, n_layerH)
        self.network = model
        self.fc1= nn.Linear(numOutputH[-1],4)
        self.fc2= nn.Linear(numOutputH[-1],4)

        
        
    
    def forward(self, xb):
        xb = self.network(xb)
        label2 = self.fc1(xb)
        label1 = self.fc2(xb)

        return {'Tool': label1, 'Action': label2}

    
    
    def training_step(self, batch,crit):
        images, l1,l2, = batch
        images, l1,l2= images.to(device), l1.to(device), l2.to(device)
        
        out = self(images)     
        
        loss1 = crit(out["Action"], l1)
        loss2 = crit(out["Tool"], l2)
        return loss1, loss2
    
    def validation_step(self, batch,crit):
        images, l1,l2 = batch
        images, l1,l2= images.to(device), l1.to(device), l2.to(device)
        
        out = self(images)     
        
        loss1 = crit(out["Action"], l1)
        loss2 = crit(out["Tool"], l2) 
        loss = loss1+loss2
        

        acc1 = accuracy(out["Action"], l1)
        acc2 = accuracy(out["Tool"], l2) 
        acc = (acc1+acc2)/2
        return {'val_loss': loss.detach(), "val_acc": acc, 'val_acc_Action': acc1, 'val_acc_Tool': acc2}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      
        batch_accs1 = [x['val_acc_Tool'] for x in outputs]
        epoch_acc1 = torch.stack(batch_accs1).mean()
        batch_accs2 = [x['val_acc_Action'] for x in outputs]
        epoch_acc2 = torch.stack(batch_accs2).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(), 'val_acc_Tool': epoch_acc1.item(), 'val_acc_Action': epoch_acc2.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}, val_acc_Tool: {:.4f}, val_acc_Action: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'],result['val_acc'],result['val_acc_Tool'],result['val_acc_Action']))
            
            
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  

def evaluate(model, val_loader,crit):
    model.eval()
    outputs = [model.validation_step(batch,crit) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD, crit =nn.CrossEntropyLoss()):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    count=0
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch in train_loader:
            optimizer.zero_grad()
            
            loss1, loss2 = model.training_step(batch,crit)
            loss = loss1+loss2
            
            train_losses.append(loss)
                     
            loss.backward()
            optimizer.step()

        
        
        result = evaluate(model, val_loader,crit)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        if result["train_loss"] - result["val_loss"] < 0:
            if count > 2:
                break
            else:
                count+=1
    
    return history

In [ ]:
def joinTensor(*args):
    l = list()
    for i in range(len(args[0])):
        newT = torch.cat((args[0][i],args[1][i]),1)
        l.append(newT)
    s = pd.Series(l)
    return s


In [ ]:
class MyDataset(Dataset):
 
    def __init__(self,df):
        df=df
 
        x=df.iloc[:,0].values
        y=df.iloc[:,1].values
        y2=df.iloc[:,2].values

        
        
 
        self.x_train=x
        self.y_train=y
        self.y2_train=y2

 
    def __len__(self):
        return len(self.y_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx],self.y2_train[idx]
    
    def getX(self):
        return self.x_train
    
    def getYA(self):
        return self.y_train
    
    def getYT(self):
        return self.y2_train


    
def plot_accuracies_losses(history,title):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(title)
    

    accuracies = [x['val_acc'] for x in history]
    ax1.plot(accuracies, '-x')
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('accuracy')
    ax1.set_title('Accuracy vs. No. of epochs')

    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    ax2.plot(train_losses, '-bx')
    ax2.plot(val_losses, '-rx')
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('loss')
    ax2.legend(['Training', 'Validation'])
    ax2.set_title('Loss vs. No. of epochs')
    
    plt.show()


In [ ]:
lateD = pd.DataFrame()
lateD[["ActionL","ToolL","Color Initial","Color Effect","Depthcolor Initial","Depthcolor Effect"]] = dataset[["ActionL","ToolL","Color Initial","Color Effect","Depthcolor Initial","Depthcolor Effect"]]
lateD["fusedC"] = joinTensor(dataset["Color Initial"],dataset["Color Effect"])
lateD["fusedD"] = joinTensor(dataset["Depthcolor Initial"],dataset["Depthcolor Effect"])

batch_size = 100

train_data, val_data = train_test_split(lateD[["fusedC","fusedD","ActionL","ToolL"]], test_size=0.3)
val_data, test_data = train_test_split(val_data,test_size=0.5)


print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

train_dataC = MyDataset(train_data[["fusedC","ActionL","ToolL"]])
val_dataC = MyDataset(val_data[["fusedC","ActionL","ToolL"]])
test_dataC = MyDataset(test_data[["fusedC","ActionL","ToolL"]])

train_dataD = MyDataset(train_data[["fusedD","ActionL","ToolL"]])
val_dataD = MyDataset(val_data[["fusedD","ActionL","ToolL"]])
test_dataD = MyDataset(test_data[["fusedD","ActionL","ToolL"]])

#load the train and validation into batches.
train_dlC = DataLoader(train_dataC, batch_size)
val_dlC = DataLoader(val_dataC, batch_size)
test_dlC = DataLoader(test_dataC, 480)

train_dlD = DataLoader(train_dataD, batch_size)
val_dlD = DataLoader(val_dataD, batch_size)
test_dlD = DataLoader(test_dataD, 480)

D 0.7270 leaky C 0.3
R 0.5390

bestC = []
bestD = []
num_epochs = [100]
opt_func = torch.optim.Adam
lrs = [0.001,0.0001]
numOutputH = [1024,512,256,128,64,32]
numOutputH1 = [1024,256,64,32,16]
numOutputH2 = [512,256,64,16]

numOutputH = numOutputH1
numOutputC = [32,64,128,256,512]

#Hyperparameter and architecture testing

for epoch in num_epochs:
    for lr in lrs:
        for n_layerC in range(2,len(numOutputC)):
            for n_layerH in range(2,len(numOutputH)):
                inH = numOutputH[:]
                inH.reverse()
                inH = inH[:n_layerH]
                inH.reverse()
                
                
                modelC = ImageLateFClassifier(numOutputC[:n_layerC], n_layerC, inH, n_layerH)
                modelD = ImageLateFClassifier(numOutputC[:n_layerC], n_layerC, inH, n_layerH)
                
                device = "cuda" if torch.cuda.is_available() else "cpu"
                print(device)
                modelC = modelC.to(device)
                
                historyC = fit(epoch, lr, modelC, train_dlC, val_dlC, opt_func)
                plot_accuracies_losses(historyC,f"RGB model Number of epochs {epoch}, Learning rate {lr}, Hidden layers {n_layerH}, Number Convolution {n_layerC}, OutputC {numOutputC[:n_layerC]}, OutputH {inH}")

                torch.cuda.empty_cache()
                
                
                modelD = modelD.to(device)
                historyD = fit(2*epoch, lr, modelD, train_dlD, val_dlD, opt_func)
                plot_accuracies_losses(historyD,f"Depth model Number of epochs {epoch*2}, Learning rate {lr}, Hidden layers {n_layerH}, Number Convolution {n_layerC}, OutputC {numOutputC[:n_layerC]}, OutputH {inH}")

                torch.cuda.empty_cache()
                
                mV = [x['val_acc'] for x in historyC][-1]
                mTL = [x['train_loss'] for x in historyC][-1]
                mVL = [x['val_loss'] for x in historyC][-1]
                mVT = [x['val_acc_Tool'] for x in historyC][-1]
                mVA = [x['val_acc_Action'] for x in historyC][-1]
                
                bestC.append((mV,mTL,mVL,mVT,mVA,"nH"+str(n_layerH),"nC"+str(n_layerC)))
                
                mV = [x['val_acc'] for x in historyD][-1]
                mTL = [x['train_loss'] for x in historyD][-1]
                mVL = [x['val_loss'] for x in historyD][-1]
                mVT = [x['val_acc_Tool'] for x in historyD][-1]
                mVA = [x['val_acc_Action'] for x in historyD][-1]
                
                bestD.append((mV,mTL,mVL,mVT,mVA,"nH"+str(n_layerH),"nC"+str(n_layerC)))

                
                
                
                


bestC.sort(key=lambda x: x[4], reverse=True)
print(bestC[0])

bestD.sort(key=lambda x: x[4],reverse=True)
print(bestD[0])

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
epoch = 100
opt_func = torch.optim.Adam
lr = 0.0001
numOutputH = [64,32,16]
numOutputC = [32,64]
batch_size = 64


modelsC = []
k=10
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

history = {'train_loss': [], 'test_loss': [],'test_acc_Tool':[],'test_acc_Action':[],'test_acc':[]}
dataset = ConcatDataset([train_dataC, val_dataC])
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

    print('Fold {}'.format(fold + 1))

    inH = numOutputH[:]
    inH.reverse()
    
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    
    model = ImageLateFClassifier(numOutputC, len(numOutputC), inH, len(inH), False)
    model.to(device)
    optimizer = torch.optim.Adam
    h = fit(epoch, lr, model, train_loader, test_loader, optimizer)
    
    modelsC.append(model)
    dif = [(x['train_loss']-x['val_loss'],x['val_acc'],x['train_loss'],x['val_loss'],x['val_acc_Tool'],x['val_acc_Action']) for x in h if x['train_loss']-x['val_loss'] > 0]
    
    history['train_loss'].append(dif[-1][2])
    history['test_loss'].append(dif[-1][3])
    history['test_acc_Tool'].append(dif[-1][4])
    history['test_acc_Action'].append(dif[-1][5])
    history['test_acc'].append(dif[-1][1])  
    torch.cuda.empty_cache()

    
    

In [ ]:
avg_train_loss = np.mean(history['train_loss'])
avg_test_loss = np.mean(history['test_loss'])
avg_test_acc = np.mean(history['test_acc'])
avg_test_acc_Tool = np.mean(history['test_acc_Tool'])
avg_test_acc_Action = np.mean(history['test_acc_Action'])

historyC = history
print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.4f} \t Average Test Loss: {:.4f}  \t Average Test Acc: {:.3f} \t Average Test Acc Tool: {:.3f} \t Average Test Acc Action: {:.3f}".format(avg_train_loss,avg_test_loss,avg_test_acc,avg_test_acc_Tool,avg_test_acc_Action))

In [ ]:
epoch = 400
opt_func = torch.optim.Adam
lr = 0.0001
numOutputH = [64,32,16]
numOutputC = [32,64]
torch.cuda.empty_cache()

modelsD = []
history = {'train_loss': [], 'test_loss': [],'test_acc_Tool':[],'test_acc_Action':[],'test_acc':[]}
dataset = ConcatDataset([train_dataD, val_dataD])
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

    print('Fold {}'.format(fold + 1))

    inH = numOutputH[:]
    inH.reverse()
    
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    
    model = ImageLateFClassifier(numOutputC, len(numOutputC), inH, len(inH), True)
    model.to(device)
    optimizer = torch.optim.Adam
    h = fit(epoch, lr, model, train_loader, test_loader, optimizer)
    
    modelsD.append(model)
    dif = [(x['train_loss']-x['val_loss'],x['val_acc'],x['train_loss'],x['val_loss'],x['val_acc_Tool'],x['val_acc_Action']) for x in h]
    
    history['train_loss'].append(dif[-1][2])
    history['test_loss'].append(dif[-1][3])
    history['test_acc_Tool'].append(dif[-1][4])
    history['test_acc_Action'].append(dif[-1][5])
    history['test_acc'].append(dif[-1][1])  
    torch.cuda.empty_cache()

In [ ]:
avg_train_loss = np.mean(history['train_loss'])
avg_test_loss = np.mean(history['test_loss'])
avg_test_acc = np.mean(history['test_acc'])
avg_test_acc_Tool = np.mean(history['test_acc_Tool'])
avg_test_acc_Action = np.mean(history['test_acc_Action'])

historyD = history
print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.4f} \t Average Test Loss: {:.4f}  \t Average Test Acc: {:.3f} \t Average Test Acc Tool: {:.3f} \t Average Test Acc Action: {:.3f}".format(avg_train_loss,avg_test_loss,avg_test_acc,avg_test_acc_Tool,avg_test_acc_Action))

In [ ]:
bestMC = max(historyC["test_acc_Tool"])
bestMD = max(historyD["test_acc_Tool"])
modelD = modelsD[historyD["test_acc_Tool"].index(bestMD)]
modelC = modelsC[historyC["test_acc_Tool"].index(bestMC)]

In [ ]:


def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    model.eval()
    
    for images, l1, l2 in loader:
        images, l1,l2 = images.to(device), l1.to(device), l2.to(device)
        
        preds = model(images)
        predsA = preds["Action"].to("cpu")
        predsT = preds["Tool"].to("cpu")
        all_predsA = torch.cat((all_preds, predsA), dim=0)
        all_predsT = torch.cat((all_preds, predsT), dim=0)
    return all_predsA, all_predsT


def get_all_combined_preds(modelC,modelD,loaderC,loaderD):
    predsAC, predsTC = get_all_preds(modelC, loaderC)
    predsAD, predsTD = get_all_preds(modelD, loaderD)
    
    
    average_resA = (predsAC+predsAD)/2
    average_resT = (predsTC+predsTD)/2
    
    
    return average_resA, average_resT

with torch.no_grad(): 
    predsAC, predsTC = get_all_preds(modelC, test_dlC)
    predsAD, predsTD = get_all_preds(modelD, test_dlD)
    
    predsAVD, predsTVD = get_all_combined_preds(modelC,modelD,test_dlC,test_dlD)



cmAC = confusion_matrix(test_dataC.getYA(), predsAC.argmax(dim=-1))
cmTC = confusion_matrix(test_dataC.getYT(), predsTC.argmax(dim=-1))
cmAD = confusion_matrix(test_dataD.getYA(), predsAD.argmax(dim=-1))
cmTD = confusion_matrix(test_dataD.getYT(), predsTD.argmax(dim=-1))
cmAVD = confusion_matrix(test_dataC.getYA(), predsAVD.argmax(dim=-1))
cmTVD = confusion_matrix(test_dataC.getYT(), predsTVD.argmax(dim=-1))

print("RGB results")
report = classification_report(test_dataC.getYA(),predsAC.argmax(dim=-1), target_names=['push', 'pull', 'left to right', "right to left"], zero_division= 0)
print('Classification Report A: ', report)

report = classification_report(test_dataC.getYT(),predsTC.argmax(dim=-1), target_names=['sshot', 'spatula', 'hook', "ruler"], zero_division= 0)
print('Classification Report T: ', report)
    

display = ConfusionMatrixDisplay(confusion_matrix=cmAC, display_labels=['push', 'pull', 'left to right', "right to left"])
display.plot()
plt.show()
display = ConfusionMatrixDisplay(confusion_matrix=cmTC, display_labels=['sshot', 'spatula', 'hook', "ruler"])
display.plot()
plt.show()


print("Depth results")
report = classification_report(test_dataD.getYA(),predsAD.argmax(dim=-1), target_names=['push', 'pull', 'left to right', "right to left"], zero_division= 0)
print('Classification Report A: ', report)

report = classification_report(test_dataD.getYT(),predsTD.argmax(dim=-1), target_names=['sshot', 'spatula', 'hook', "ruler"], zero_division= 0)
print('Classification Report T: ', report)
    

display = ConfusionMatrixDisplay(confusion_matrix=cmAD, display_labels=['push', 'pull', 'left to right', "right to left"])
display.plot()
plt.show()
display = ConfusionMatrixDisplay(confusion_matrix=cmTD, display_labels=['sshot', 'spatula', 'hook', "ruler"])
display.plot()
plt.show()

print("Average results")
report = classification_report(test_dataC.getYA(),predsAVD.argmax(dim=-1), target_names=['push', 'pull', 'left to right', "right to left"], zero_division= 0)
print('Classification Report A: ', report)

report = classification_report(test_dataC.getYT(),predsTVD.argmax(dim=-1), target_names=['sshot', 'spatula', 'hook', "ruler"], zero_division= 0)
print('Classification Report T: ', report)
    

display = ConfusionMatrixDisplay(confusion_matrix=cmAVD, display_labels=['push', 'pull', 'left to right', "right to left"])
display.plot()
plt.show()
display = ConfusionMatrixDisplay(confusion_matrix=cmTVD, display_labels=['sshot', 'spatula', 'hook', "ruler"])
display.plot()
plt.show()